In [ ]:
# Install required packages (run once)
!pip install pandas openpyxl sqlalchemy psycopg2-binary


In [ ]:
import os
import re
import pandas as pd
from datetime import datetime
from sqlalchemy import create_engine, text


In [ ]:
user = "postgres"
password = "*******"
host = "localhost"
port = "5432"
database = "knee_database1"

engine = create_engine(
    f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}"
)

# Confirm connection
with engine.connect() as connection:
    db = connection.execute(text("SELECT current_database();")).scalar()
    print(f"✅ Connected to database: {db}")


In [ ]:
column_map = {
    'Age': 'age',
    'GENDER': 'gender',
    'Weight (lbs)': 'weight_lbs',
    'Weight (kg)': 'weight_kg',
    'Height (in)': 'height_in',
    'Height (cm)': 'height_cm',
    'KneeInjury': 'knee_injury',
    'DomLimb': 'dom_limb',
    'SurgeryLimb': 'surgery_limb',
    'TimeSinceSx': 'time_since_sx',
    'Date': 'session_date'
}


In [ ]:
subject_fields = [
    'age', 'gender', 'weight_lbs', 'weight_kg',
    'height_in', 'height_cm',
    'knee_injury', 'dom_limb', 'surgery_limb'
]

meta_cols = subject_fields + ['time_since_sx', 'session_date']


In [ ]:
existing_measurements = pd.read_sql(
    "SELECT name, measurement_id FROM measurement",
    con=engine
).set_index('name')['measurement_id'].to_dict()

loaded_files = pd.read_sql(
    "SELECT file_name FROM loaded_files",
    con=engine
)['file_name'].tolist()


In [ ]:
for file in os.listdir(folder_path):

    if not file.endswith(".xlsx") or not re.match(r"^KneeSx\d+(?:-\d)?", file):
        continue

    if file in loaded_files:
        continue

    subject_id = re.match(r"^KneeSx(\d+(?:-\d)?)", file).group(1)
    file_path = os.path.join(folder_path, file)

    print(f"📄 Processing {file}")

    # --- Read Excel ---
    df = pd.read_excel(file_path)
    df.columns = df.columns.str.strip()

    # ======================================================
    # 🔥🔥🔥 COLUMN MAP IS APPLIED RIGHT HERE 🔥🔥🔥
    # ======================================================
    df = df.rename(columns=column_map)
    # ======================================================

    # --- Insert Subject ---
    subject_data = df.loc[0, subject_fields].to_frame().T
    subject_data.insert(0, 'subject_id', subject_id)

    subject_data.to_sql(
        'subject',
        con=engine,
        if_exists='append',
        index=False,
        method='multi'
    )

    # --- Insert Test Session ---
    session_df = pd.DataFrame([{
        'subject_id': subject_id,
        'session_date': df.loc[0, 'session_date'],
        'time_since_sx': df.loc[0, 'time_since_sx']
    }])

    session_df.to_sql(
        'test_session',
        con=engine,
        if_exists='append',
        index=False,
        method='multi'
    )

    session_id = pd.read_sql(
        """
        SELECT session_id
        FROM test_session
        WHERE subject_id = %s
        ORDER BY session_id DESC
        LIMIT 1
        """,
        con=engine,
        params=(subject_id,)
    )['session_id'][0]

    # --- Insert Measurements ---
    measurement_cols = [c for c in df.columns if c not in meta_cols]

    for col in measurement_cols:

        if col not in existing_measurements:
            with engine.begin() as conn:
                conn.execute(
                    text("INSERT INTO measurement (name) VALUES (:name)"),
                    {"name": col}
                )

            measurement_id = pd.read_sql(
                "SELECT measurement_id FROM measurement WHERE name = %s",
                con=engine,
                params=(col,)
            )['measurement_id'][0]

            existing_measurements[col] = measurement_id
        else:
            measurement_id = existing_measurements[col]

        sm_df = pd.DataFrame([{
            'session_id': session_id,
            'measurement_id': measurement_id,
            'value': df.loc[0, col]
        }])

        sm_df.to_sql(
            'session_measurement',
            con=engine,
            if_exists='append',
            index=False,
            method='multi'
        )

    # --- Log File ---
    log_df = pd.DataFrame([{
        'file_name': file,
        'upload_date': datetime.now()
    }])

    log_df.to_sql(
        'loaded_files',
        con=engine,
        if_exists='append',
        index=False
    )

    print(f"✅ Imported {file}")
